In [8]:
from langchain.document_loaders import CSVLoader,PyPDFLoader,Docx2txtLoader
from langchain.embeddings import OllamaEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
import os

In [ ]:
print(os.listdir(os.getcwd()))

In [ ]:
file_paths = os.listdir(os.getcwd())
file_paths.pop(0)
file_paths.pop(0)

In [ ]:
combined_docs = []

for path in file_paths:
    
    file_format = os.path.splitext(path)[1].lower()

    if file_format==".pdf":
        print("Loading PDF")
        pdf_loader = PyPDFLoader(path)
        combined_docs.extend(pdf_loader.load())
    elif file_format==".csv":
        print("Loading CSV ")
        csv_loader = CSVLoader(path)
        combined_docs.extend(csv_loader.load())
    elif file_format==".docx":
        print("Loading DOCX")
        docx_loader = Docx2txtLoader(path)
        combined_docs.extend(docx_loader.load())
    else:
        print("File format is not supported")
    
   

In [12]:
splitter = RecursiveCharacterTextSplitter(chunk_size=2000,chunk_overlap=200)
splitted_docs = splitter.split_documents(combined_docs)

In [13]:
embedding_model = OllamaEmbeddings(model="nomic-embed-text:latest")
db = Chroma.from_documents(splitted_docs , embedding_model, persist_directory=None)
retriever = db.as_retriever()

In [20]:
from langchain_community.llms import ollama
from langchain.prompts import PromptTemplate

In [21]:
prompt = PromptTemplate(
    input_variables=["context" , "question"],
    template="You are an professional assistant,\
        based on what the user asks you should give a proper answer. \
            Make use of the context to generate an accurate answer. \
                <context>{context}</context> \
                    <question>{question}</question>"
)
llm = ollama.Ollama(model="mistral:latest")

C:\Users\hp\AppData\Local\Temp\ipykernel_19036\2275093053.py:9: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  llm = ollama.Ollama(model="mistral:latest")


In [22]:
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(llm=llm,retriever=retriever,chain_type_kwargs={"prompt":prompt})




In [23]:
result = qa_chain.invoke("What is LLM")
result["result"]

' Language Learning Model (LLM) is a type of artificial intelligence model that is trained to generate human-like text. It learns patterns and structures in language by processing vast amounts of text data, such as books, websites, and conversation transcripts. The model uses techniques like transformers, autoregressive models, or encoder-decoders to predict the next word in a sequence based on the context provided. These models can be fine-tuned for various tasks, such as text generation, summarization, question answering, and more.'

Language Learning Model (LLM) is a type of artificial intelligence model that is trained to generate human-like text. It learns patterns and structures in language by processing vast amounts of text data, such as books, websites, and conversation transcripts. The model uses techniques like transformers, autoregressive models, or encoder-decoders to predict the next word in a sequence based on the context provided. These models can be fine-tuned for various tasks, such as text generation, summarization, question answering, and more.
